In [1]:
import pandas as pd
import numpy as np
import pickle
from scipy import stats

In [5]:
# create combined dataframe of expert + model predictions
human_label_path = 'nih_full_raw_data/all_findings_expert_labels/test_individual_readers.csv'
n_experts = 5
annotator_id_cols = [4343880271, 4343882583, 4343882785, 4343883593, 4343883996]
human_labels = pd.read_csv(human_label_path)
human_labels.reset_index(drop=True, inplace=True)
annotations = human_labels.pivot(
    index='Image ID', columns='Reader ID', values='Abnormal'
).reset_index()

m = pd.read_csv("cxr_model_predictions.csv", index_col=0)
m['img_name'] = m['img_name'].apply(lambda x: x[13:])
m['score'] = m['score'].apply(lambda x: float(x[1:-1]))

df = annotations.merge(m, left_on='Image ID', right_on='img_name')
df = df.drop('Image ID', axis=1)
df['Y_H'] = df.apply(lambda x: [x[c] for c in annotator_id_cols], axis=1)
df['consensus'] = df['Y_H'].apply(lambda x: stats.mode(x, keepdims=True)[0][0])
df['model_p0'] = 1-df['score']
df.columns = [
    'expert1', 'expert2', 'expert3', 'expert4', 'expert5', 'img_name',
                'model_p1', 'Y_H', 'consensus','model_p0'
]
df = df[[
    'expert1', 'expert2', 'expert3', 'expert4', 'expert5', 'img_name',
                'model_p0', 'model_p1', 'Y_H', 'consensus'
]]
df.to_csv('data_clean.csv', index=None)
df.head()

,expert1,expert2,expert3,expert4,expert5,img_name,model_p0,model_p1,Y_H,consensus
0,1,1,1,1,1,00000211_006.png,0.000668,0.999332,"[1, 1, 1, 1, 1]",1
1,1,1,1,1,1,00000211_022.png,0.013090,0.986910,"[1, 1, 1, 1, 1]",1
2,1,1,1,1,1,00000211_028.png,0.006696,0.993304,"[1, 1, 1, 1, 1]",1
3,1,1,1,1,1,00000211_029.png,0.000518,0.999482,"[1, 1, 1, 1, 1]",1
4,1,1,1,1,1,00000218_002.png,0.000456,0.999544,"[1, 1, 1, 1, 1]",1
